# Mask R-CNN for detecting car damage

### Table of content

1. Use case description
2. Data
3. Model
4. Training
5. Accuracy
6. Results

### Use case description

Mask R-CNN is an instance segmentation where each individual object is separated and identified; this approach incldues finding the bounding boxes of each object and assignment of the unique label to the identified objects [1]. The model is trained on COCO dataset which is a large-scale object detection, segmentation, and captioning dataset including 80 categories, 1,500,000 object instances, and over 330,000 images [2]. 

The goal of this notebook is to explore the Mask R-CNN model to detect car damage areas. The underlying rationale for this model development resides in its potential utility within the insurance industry, wherein expedited claims processing can be facilitated through the integration of user-uploaded images, thereby enabling damage assessment. Additionally, the model exhibits relevance to financial institutions, particularly when engaged in the underwriting of automotive loans, with a specific focus on pre-owned vehicles.

#### References

[1] He, K., Gkioxari, G., Dollár, P., & Girshick, R. (2017). Mask r-cnn. In *Proceedings of the IEEE international conference on computer vision* (pp. 2961-2969).

[2] Lin, T. Y., Maire, M., Belongie, S., Hays, J., Perona, P., Ramanan, D., ... & Zitnick, C. L. (2014). Microsoft coco: Common objects in context. In *Computer Vision–ECCV 2014: 13th European Conference, Zurich, Switzerland, September 6-12, 2014, Proceedings, Part V 13* (pp. 740-755). Springer International Publishing.